
## Import necessary libraries

In [1]:
import torch

# Verifica si CUDA (GPU) está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [2]:
# Python libraries
import datetime as dt
import re
import pickle
from tqdm.notebook import tqdm
import os
import sys
import time
import logging
import random
from collections import defaultdict, Counter

# Data Science modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('ggplot')


# Import nltk modules and download dataset
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop = set(stopwords.words('english'))

# Import Pytorch modules
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.autograd import Variable
from torch.optim import Adam, AdamW

C:\Users\luisl\AppData\Local\Temp\ipykernel_5140\269888456.py:15: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luisl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
import warnings
warnings.filterwarnings("ignore")

## cargar texto resumido 

In [4]:
import pickle

# Define the path to the pickle file (same as before)
ruta_pickle = 'C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/data/preprocessed/text_resumido_v2.pkl'

# Open the file in binary read mode
with open(ruta_pickle, 'rb') as pickle_file:
  # Use pickle.load to deserialize the data and create a new DataFrame
  loaded_df = pickle.load(pickle_file)

print("¡DataFrame cargado del archivo pickle exitosamente!")

¡DataFrame cargado del archivo pickle exitosamente!


In [5]:
loaded_df.head()

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary
13691,speech,2023-10-02,Monetary Policy and Financial Stability,Vice Chair for Supervision Michael S. Barr,3267,<NA>,NaN,2023-11-01,0,5.25,Thank you for the opportunity to speak to you....,I am squarely focused on our dual mandate to p...
13692,speech,2023-10-02,Brief Remarks on the Economy and Bank Regulation,Governor Michelle W. Bowman,1996,<NA>,NaN,2023-11-01,0,5.25,Thank you for the invitation to join you today...,Fed Chair Janet Yellen speaks to community ban...
13693,speech,2023-10-04,"The Role of Research, Data, and Analysis in Ba...",Governor Michelle W. Bowman,3757,<NA>,NaN,2023-11-01,0,5.25,It's a pleasure to be back in St. Louis for th...,The annual Community Banking Research Conferen...
13694,speech,2023-10-07,Brief Remarks on the Economy and Insights from...,Governor Michelle W. Bowman,2248,<NA>,NaN,2023-11-01,0,5.25,It is a pleasure to be with you here today.1 A...,"""I would like to share some thoughts about tha..."
13695,speech,2023-10-09,Capital Supports Lending,Vice Chair for Supervision Michael S. Barr,1755,<NA>,NaN,2023-11-01,0,5.25,Thank you for the opportunity to join you toda...,Federal Reserve Chairman Janet Yellen speaks o...


In [6]:
train_sent_df = loaded_df

In [7]:
import pandas as pd

# Porcentaje deseado de las filas (e.g., 10%)
porcentaje = 100

# Calcula el número de filas a seleccionar basado en el porcentaje
num_filas = int(len(train_sent_df) * (porcentaje / 100))

# Selecciona aleatoriamente el porcentaje deseado de filas del DataFrame
submuestra_df = train_sent_df.sample(n=num_filas)

submuestra_df = submuestra_df.dropna(subset=['next_decision'])

# Convierte la columna deseada a una lista
X_submuestra = submuestra_df['summary'].to_list()



carga del primer modelo reentrenado

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Define labels
labels = ['negative', 'neutral', 'positive']

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# Define the path to the saved model (assuming the model is saved in a folder)
model_path = "C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/models/best_model_5e-05_64_V2.pt"  # Replace with your actual path

# Function to preprocess text
def preprocess_text(text, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs.input_ids.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    attention_mask = inputs.attention_mask.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return input_ids, attention_mask

# Load the model
model_finbert = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model_finbert.load_state_dict(torch.load(model_path))
model_finbert.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))  # Move to GPU if available

with torch.no_grad():
    model_finbert.eval()

    # Example usage
    text = "The stock market is booming!"
    input_ids, attention_mask = preprocess_text(text, tokenizer)

    # Get predictions
    outputs = model_finbert(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

    # Get the predicted sentiment
    predicted_sentiment = labels[predicted_label]

    print(f"Predicted sentiment for '{text}': {predicted_sentiment}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predicted sentiment for 'The stock market is booming!': positive


twitter-roberta-base-sentiment Model¶


In [9]:
import torch
import numpy as np
import scipy.special
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model_roberta = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model_roberta.load_state_dict(torch.load(model_path))
model_roberta.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_roberta.to(device)

tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}
labels = ['negative', 'neutral', 'positive']

# Asumiendo que X_submuestra es tu lista de textos a analizar
preds_roberta = []
preds_proba_roberta = []

for x in tqdm(X_submuestra, desc="Procesando predicciones"):
    with torch.no_grad():
        # Tokenizar y mover el texto completo a la GPU
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs).to(device)
        # Obtener las logits del modelo
        logits = model_roberta(**input_sequence).logits
        # Aplicar softmax en la GPU
        scores = torch.softmax(logits, dim=1).cpu().numpy().squeeze()
    
    sentimiento_general = labels[np.argmax(scores)]
    probabilidad_general = np.max(scores)
    
    preds_roberta.append(sentimiento_general)
    preds_proba_roberta.append(probabilidad_general)


Procesando predicciones: 100%|██████████| 500/500 [00:12<00:00, 41.37it/s]


In [10]:
submuestra_df['predicted_roberta'] = preds_roberta; submuestra_df

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary,predicted_roberta
13900,testimony,2004-05-13,U.S. - EU regulatory dialogue,Governor Susan Schmidt Bies,3067,<NA>,NaN,2004-06-30,1,1.25,"Thank you, Mr. Chairman, for the opportunity t...",The Dialogue was initiated by the Treasury Dep...,positive
13934,testimony,2006-07-19,Semiannual Monetary Policy Report to the Congress,Ben Bernanke,2356,<NA>,NaN,2006-08-08,0,5.25,"Over the period since our February report, the...",U.S. economy appears to be in a period of tran...,positive
13947,testimony,2007-05-21,"Bank mergers, Community Reinvestment Act enfor...","Sandra Braunstein, Director, Division of Consu...",5460,<NA>,NaN,2007-06-28,0,5.25,"Chairman Kucinich, Ranking Member Issa, and me...","""I appreciate this opportunity to appear in Cl...",neutral
14043,testimony,2010-07-21,Semiannual Monetary Policy Report to the Congress,Ben Bernanke,3703,<NA>,NaN,2010-08-10,0,0.00,Economic and Financial DevelopmentsThe economi...,The economic expansion that began in the middl...,positive
13921,testimony,2005-09-08,Commodity Futures Modernization Act of 2000,"Patrick Parkinson, Deputy Director, Division o...",1725,<NA>,NaN,2005-09-20,1,3.75,"Chairman Shelby, Senator Sarbanes, and members...",The Federal Reserve Board believes that the CF...,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13913,testimony,2005-04-21,Budget process reforms,Alan Greenspan,1547,<NA>,NaN,2005-05-03,1,3.00,The U.S. economy delivered a solid performance...,U.S. budget ran a deficit equal to about 3-1/2...,positive
13944,testimony,2007-03-27,Subprime mortgages,"Sandra F. Braunstein, Director, Division of Co...",6375,<NA>,NaN,2007-05-09,0,5.25,"Madam Chair Maloney, Ranking Member Gillmor, m...",The banking system has changed dramatically si...,negative
14165,testimony,2020-09-22,"Coronavirus Aid, Relief, and Economic Security...",Jerome Powell,2400,<NA>,NaN,2020-11-05,0,0.00,"Chairwoman Waters, Ranking Member McHenry, and...","The Federal Reserve, along with others across ...",negative
14163,testimony,2020-09-08,Diversity and Inclusion,"Testimony by Sheila Clark, Program Director, O...",483,<NA>,NaN,2020-09-16,0,0.00,"Chair Beatty, Ranking Member Wagner, and membe...",The Board of Governors of the Federal Reserve ...,neutral


In [49]:
submuestra_df.head()

,type,date,title,speaker,word_count,decision,rate,next_meeting,next_decision,next_rate,text,summary,predicted_roberta
13900,testimony,2004-05-13,U.S. - EU regulatory dialogue,Governor Susan Schmidt Bies,3067,<NA>,NaN,2004-06-30,1,1.25,"Thank you, Mr. Chairman, for the opportunity t...",The Dialogue was initiated by the Treasury Dep...,positive
13934,testimony,2006-07-19,Semiannual Monetary Policy Report to the Congress,Ben Bernanke,2356,<NA>,NaN,2006-08-08,0,5.25,"Over the period since our February report, the...",U.S. economy appears to be in a period of tran...,positive
13947,testimony,2007-05-21,"Bank mergers, Community Reinvestment Act enfor...","Sandra Braunstein, Director, Division of Consu...",5460,<NA>,NaN,2007-06-28,0,5.25,"Chairman Kucinich, Ranking Member Issa, and me...","""I appreciate this opportunity to appear in Cl...",neutral
14043,testimony,2010-07-21,Semiannual Monetary Policy Report to the Congress,Ben Bernanke,3703,<NA>,NaN,2010-08-10,0,0.00,Economic and Financial DevelopmentsThe economi...,The economic expansion that began in the middl...,positive
13921,testimony,2005-09-08,Commodity Futures Modernization Act of 2000,"Patrick Parkinson, Deputy Director, Division o...",1725,<NA>,NaN,2005-09-20,1,3.75,"Chairman Shelby, Senator Sarbanes, and members...",The Federal Reserve Board believes that the CF...,positive


In [55]:
#selected_columns_df = submuestra_df[['date', 'predicted_roberta', 'next_meeting','next_decision']]

Cargar datos numericos

In [56]:
# Load previously processed non-text data
# Load data
file = open('C:/Users/luisl/OneDrive/Escritorio/TFM_IA3/tfm_ia3_nodata/data/train_data/nontext_train_small.pickle', 'rb')
train_df = pickle.load(file)
file.close()

print(train_df.shape)
train_df

(403, 13)


,target,RateDiff,prev_decision,Employ_diff_year,Rsales_diff_year,Unemp_diff_year,Employ_diff_prev,GDP_diff_year,PCE_diff_prev,Unemp_diff_prev,Hsales_diff_year,GDP_diff_prev,Taylor
date,,,,,,,,,,,,,
1982-10-05,-1,-0.50,0,-2294.0,2.004317,-2.5,-180.0,-1.010745,0.460401,-0.3,42.307692,0.456177,11.061999
1982-11-16,-1,-0.50,-1,-2473.0,2.004317,-2.5,-276.0,-2.556042,0.370245,-0.3,34.831461,-0.382267,10.045779
1982-12-21,0,0.00,-1,-2385.0,2.004317,-2.5,-121.0,-2.556042,0.634292,-0.4,45.026178,-0.382267,10.105638
1983-01-14,0,0.00,0,-2124.0,2.004317,-2.3,-15.0,-2.556042,0.368789,0.0,14.004376,-0.382267,9.766001
1983-01-21,0,0.00,0,-2124.0,2.004317,-2.3,-15.0,-2.556042,0.368789,0.0,14.004376,-0.382267,9.766001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-14,0,0.00,1,3928.0,-1.938580,-0.1,281.0,1.717927,0.306901,-0.3,11.635220,0.556381,12.123858
2023-07-26,1,0.25,0,3663.0,-1.573436,0.0,105.0,1.717927,0.288262,0.1,21.314387,0.556381,12.011106
2023-09-20,0,0.00,1,3144.0,-0.840916,-0.1,165.0,2.382468,0.119978,-0.3,2.507837,0.511122,11.295164


In [57]:
train_df.reset_index(inplace=True)

In [58]:
train_df.columns

Index(['date', 'target', 'RateDiff', 'prev_decision', 'Employ_diff_year',
       'Rsales_diff_year', 'Unemp_diff_year', 'Employ_diff_prev',
       'GDP_diff_year', 'PCE_diff_prev', 'Unemp_diff_prev', 'Hsales_diff_year',
       'GDP_diff_prev', 'Taylor'],
      dtype='object')

In [59]:
doc_types = submuestra_df['type'].unique()

merged_dict = defaultdict(list)

for i, row in train_df.iterrows():
    text_rows = submuestra_df.loc[submuestra_df['next_meeting'] == row['date']]
    merged_text_all = ""
    for doc_type in doc_types:
        merged_text = ""
        for text in text_rows.loc[text_rows['type'] == doc_type]['text']:
            merged_text += " " + text
        merged_dict[doc_type].append(merged_text)
        merged_text_all += merged_text
    merged_dict['text'].append(merged_text_all)
    
    # Append 'next_decision' and 'predicted_roberta' only if text_rows is not empty
    if not text_rows.empty:
        merged_dict['next_decision'].append(text_rows['next_decision'].iloc[0])  # Assuming only one decision per meeting
        merged_dict['predicted_roberta'].append(text_rows['predicted_roberta'].iloc[0])
    else:
        merged_dict['next_decision'].append(None)  # Append None if no matching rows found
        merged_dict['predicted_roberta'].append(None)
    
for key in merged_dict.keys():
    train_df[key] = merged_dict[key]

print(train_df)

          date  target  RateDiff  prev_decision  Employ_diff_year  \
0   1982-10-05      -1     -0.50              0           -2294.0   
1   1982-11-16      -1     -0.50             -1           -2473.0   
2   1982-12-21       0      0.00             -1           -2385.0   
3   1983-01-14       0      0.00              0           -2124.0   
4   1983-01-21       0      0.00              0           -2124.0   
..         ...     ...       ...            ...               ...   
398 2023-06-14       0      0.00              1            3928.0   
399 2023-07-26       1      0.25              0            3663.0   
400 2023-09-20       0      0.00              1            3144.0   
401 2023-11-01       0      0.00              0            3056.0   
402 2023-12-13       0      0.00              0            2791.0   

     Rsales_diff_year  Unemp_diff_year  Employ_diff_prev  GDP_diff_year  \
0            2.004317             -2.5            -180.0      -1.010745   
1            2.004317

In [63]:
train_df.tail()

,date,target,RateDiff,prev_decision,Employ_diff_year,Rsales_diff_year,Unemp_diff_year,Employ_diff_prev,GDP_diff_year,PCE_diff_prev,Unemp_diff_prev,Hsales_diff_year,GDP_diff_prev,Taylor,testimony,speech,text,next_decision,predicted_roberta
398,2023-06-14,0,0.00,1,3928.0,-1.938580,-0.1,281.0,1.717927,0.306901,-0.3,11.635220,0.556381,12.123858,"Chairman McHenry, Ranking Member Waters, and ...",,"Chairman McHenry, Ranking Member Waters, and ...",0.0,neutral
399,2023-07-26,1,0.25,0,3663.0,-1.573436,0.0,105.0,1.717927,0.288262,0.1,21.314387,0.556381,12.011106,"In my work as a Federal Reserve governor, my ...",,"In my work as a Federal Reserve governor, my ...",1.0,neutral
400,2023-09-20,0,0.00,1,3144.0,-0.840916,-0.1,165.0,2.382468,0.119978,-0.3,2.507837,0.511122,11.295164,,,,NaN,None
401,2023-11-01,0,0.00,0,3056.0,0.295139,-0.3,262.0,2.926887,0.329011,0.0,23.456790,1.193871,10.970821,,It's a pleasure to join you today to discuss ...,It's a pleasure to join you today to discuss ...,0.0,neutral
402,2023-12-13,0,0.00,0,2791.0,0.941219,-0.1,199.0,2.926887,0.148529,0.2,1.374570,1.193871,10.463508,"Chairman Brown, Ranking Member Scott, and oth...",It is a pleasure to be with you in Columbus t...,"Chairman Brown, Ranking Member Scott, and oth...",0.0,neutral


In [61]:
# Assuming df is your DataFrame and 'column_name' is the column where you want to check for NaN values
nan_count = train_df[train_df['next_decision'].isna()].shape[0]

print("Number of rows with NaN values in 'column_name':", nan_count)

Number of rows with NaN values in 'column_name': 232
